In [ ]:
!pip install tqdm
!pip install opencage
!pip install scikit-learn

In [2]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm
from opencage.geocoder import OpenCageGeocode
from google.colab import userdata
import requests
import os
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
csv_path = '/content/drive/MyDrive/ML_Dataset/AQI_2010_2023/stations_info.csv'
df = pd.read_csv(csv_path)
df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,APPCB,"Tirumala, Tirupati",July,7,2016
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017
4,AP005,Andhra Pradesh,Amaravati,APPCB,"Secretariat, Amaravati",November,11,2017


In [ ]:
df['station_address'] = df['station_location'] + ', ' +df['state']
df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address
0,AP001,Andhra Pradesh,Tirupati,APPCB,"Tirumala, Tirupati",July,7,2016,"Tirumala, Tirupati , Andhra Pradesh"
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh"
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh"
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr..."
4,AP005,Andhra Pradesh,Amaravati,APPCB,"Secretariat, Amaravati",November,11,2017,"Secretariat, Amaravati , Andhra Pradesh"


In [ ]:
def compute_latitude_longitude(row):
  try:
    location = geolocator.geocode(row['station_address'])
    time.sleep(3)
    if location:
        return pd.Series({'latitude': location.latitude, 'longitude': location.longitude})
    else:
        return pd.Series({'latitude': None, 'longitude': None})
  except:
    return pd.Series({'latitude': None, 'longitude': None})

In [ ]:
geolocator = Nominatim(user_agent="pollution_station_locator")
tqdm.pandas()
df[['latitude', 'longitude']] = df.progress_apply(compute_latitude_longitude, axis=1)

  0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon.csv', index=False)

Loading intermediate data for latitude and longitude

In [ ]:
csv_path = '/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon.csv'
df = pd.read_csv(csv_path)
retry_df = df[df['latitude'].isna()]
retry_df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address,latitude,longitude
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh",NaN,NaN
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh",NaN,NaN
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr...",NaN,NaN
7,AP008,Andhra Pradesh,Tirupati,APPCB,"Vaikuntapuram, Tirupati",November,11,2022,"Vaikuntapuram, Tirupati , Andhra Pradesh",NaN,NaN
9,AP010,Andhra Pradesh,Kadapa,APPCB,"Yerramukkapalli, Kadapa",January,1,2023,"Yerramukkapalli, Kadapa , Andhra Pradesh",NaN,NaN


Alternate method to get latitude and longitude

In [ ]:
geocoder = OpenCageGeocode(userdata.get('OPENCAGE_API_KEY'))
def compute_latitude_longitude(row):
  try:
    query = row['station_address'] + ", India"
    result = geocoder.geocode(query)
    time.sleep(3)
    if result and len(result):
        return pd.Series({'latitude': result[0]['geometry']['lat'], 'longitude': result[0]['geometry']['lng']})
    else:
        return pd.Series({'latitude': None, 'longitude': None})
  except:
    return pd.Series({'latitude': None, 'longitude': None})

In [ ]:
tqdm.pandas()
retry_df[['latitude', 'longitude']] = retry_df.progress_apply(compute_latitude_longitude, axis=1)

  0%|          | 0/230 [00:00<?, ?it/s]

/tmp/ipython-input-8-3659275104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retry_df[['latitude', 'longitude']] = retry_df.progress_apply(compute_latitude_longitude, axis=1)


In [ ]:
retry_df

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address,latitude,longitude
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh",16.51928,80.63049
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh",17.68009,83.20161
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr...",17.00517,81.77784
7,AP008,Andhra Pradesh,Tirupati,APPCB,"Vaikuntapuram, Tirupati",November,11,2022,"Vaikuntapuram, Tirupati , Andhra Pradesh",13.63551,79.41989
9,AP010,Andhra Pradesh,Kadapa,APPCB,"Yerramukkapalli, Kadapa",January,1,2023,"Yerramukkapalli, Kadapa , Andhra Pradesh",14.47750,78.82353
...,...,...,...,...,...,...,...,...,...,...,...
441,WB003,West Bengal,Howrah,WBPCB,"Ghusuri, Howrah",June,6,2016,"Ghusuri, Howrah , West Bengal",22.61667,88.35000
442,WB004,West Bengal,Durgapur,WBPCB,"Sidhu Kanhu Indoor Stadium, Durgapur",July,7,2016,"Sidhu Kanhu Indoor Stadium, Durgapur , West Be...",23.49957,87.32155
443,WB005,West Bengal,Kolkata,WBPCB,"Rabindra Bharati University, Kolkata",September,9,2016,"Rabindra Bharati University, Kolkata , West Be...",22.56263,88.36304
445,WB007,West Bengal,Asansol,WBPCB,"Asansol Court Area, Asansol",February,2,2018,"Asansol Court Area, Asansol , West Bengal",23.68333,86.98333


In [ ]:
merged_df = pd.concat([df.dropna(subset=['latitude', 'longitude']), retry_df]).drop_duplicates(subset=['file_name'])
merged_df = merged_df.sort_values(by='file_name')
merged_df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon_final.csv', index=False)

Adding elevation information

In [ ]:
def compute_elevation(row):
  try:
    latitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={latitude},{longitude}"
    response = requests.get(url).json()
    time.sleep(3)
    if response:
        return response['results'][0]['elevation']
    else:
        return None
  except:
    return None

In [ ]:
tqdm.pandas()
merged_df['elevation'] = merged_df.progress_apply(compute_elevation, axis=1)

  0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon_ele.csv', index=False)

Clean and consolidating individual AQI files

In [32]:
folder_path = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/renamed_column/"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv") and not f.endswith('stations_info.csv')]
file_headers = defaultdict(set)
file_header_counter = {}
for file in tqdm(csv_files):
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path, nrows=0)
        for column in df.columns:
              file_headers[column].add(file)
    except Exception as e:
        print(f"Error reading {file}: {e}")
file_headers = {k: sorted(list(v)) for k, v in file_headers.items()}
header_df = pd.DataFrame([
    {"header": header, "files": files, "file_count": len(files)}
    for header, files in file_headers.items()
])

  0%|          | 0/453 [00:00<?, ?it/s]

In [33]:
header_df = header_df.sort_values(by='file_count', ascending= False)
header_df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/header_presence_v2.csv', index=False)

In [ ]:
# hedaer_df['header'].values.tolist()

vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
X = vectorizer.fit_transform(header_df['header'].values.tolist())
cosine_sim = cosine_similarity(X)
threshold = 1 - 0.2
clustering = AgglomerativeClustering(
    # affinity='precomputed',
    linkage='complete',
    distance_threshold=threshold,
    n_clusters=None
)
labels = clustering.fit_predict(1 - cosine_sim)
clusters = {}
for idx, label in enumerate(labels):
    clusters.setdefault(label, []).append(header_df['header'].values.tolist()[idx])

# clusters_df = pd.DataFrame([
#     {"header": header, "files": files, "file_count": len(files)}
#     for header, files in file_headers.items()
# ])
# # Display result
for cluster_id, items in clusters.items():
    print(f"Cluster {cluster_id + 1}: {items}")


In [28]:
def column_stats(df, column_name, filename):
  print(f"Stats for file: {filename}")
  if column_name not in df.columns:
    print(f"Column '{column_name}' not found in the DataFrame.")
    return
  cleaned_column = df[column_name].dropna()
  if cleaned_column.empty:
    print(f"Column '{column_name}' contains no valid numerical entries.")
    return
  column_range_max = cleaned_column.max()
  column_range_min = cleaned_column.min()
  column_avg = cleaned_column.mean()
  print(f"Column: {column_name}")
  print(f"Range (excluding nulls): {column_range_max:.2f} - {column_range_min:.2f}")
  print(f"Average (excluding nulls): {column_avg:.2f}")

In [21]:
file_paths = ['CG003.csv']
folder_path = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023/"

for path in file_paths:
  column_stats(pd.read_csv(folder_path + path), 'CO (mg/Nm3)', path)
  print("Stat complete")

Stats for file: CG003.csv
Column: CO (mg/Nm3)
Range (excluding nulls): 2.97 - 0.19
Average (excluding nulls): 0.74
Stat complete


Column rename for calculating AQI Index for each hour

In [31]:
input_folder = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023/"
output_folder = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/renamed_column/"
os.makedirs(output_folder, exist_ok=True)
rename_map = {
    "CO (ng/m3)": "CO (mg/m3)",
    "CO (ug/m3)": "CO (mg/m3)",
    "CO (mg/Nm3)": "CO (mg/m3)",
    "SO2 ()": "SO2 (ug/m3)",
    "Ozone ()": "Ozone (ug/m3)",
    "Ozone (ppb)": "Ozone (ug/m3)",
    "NH3 ()": "NH3 (ug/m3)",
    "NH3 (ppb)": "NH3 (ug/m3)"
}
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv") and not f.endswith('stations_info.csv')]
for file in tqdm(csv_files, desc="Processing CSV files"):
    input_path = os.path.join(input_folder, file)
    output_path = os.path.join(output_folder, file)
    try:
        df = pd.read_csv(input_path)
        df.rename(columns=rename_map, inplace=True)
        df.to_csv(output_path, index=False)
    except Exception as e:
        tqdm.write(f"Error processing {file}: {e}")


Processing CSV files:   0%|          | 0/453 [00:00<?, ?it/s]

List files which are missing minimum AQI fields

In [34]:
folder_path = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/renamed_column/"
required_columns = [
    "NO2 (ug/m3)",
    "PM2.5 (ug/m3)",
    "PM10 (ug/m3)",
    "CO (mg/m3)",
    "SO2 (ug/m3)",
    "Ozone (ug/m3)",
    "NH3 (ug/m3)"
]
missing_columns_report = []
for file in tqdm(os.listdir(folder_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path, nrows=1)
            missing = [col for col in required_columns if col not in df.columns]
            if missing:
                missing_columns_report.append({
                    "file": file,
                    "missing_columns": missing
                })
        except Exception as e:
            print(f"Error reading {file}: {e}")
if missing_columns_report:
    print("\nFiles missing required AQI columns:\n")
    for entry in missing_columns_report:
        print(f"{entry['file']} → Missing: {', '.join(entry['missing_columns'])}")
else:
    print("All CSV files contain the required columns.")

  0%|          | 0/453 [00:00<?, ?it/s]


Files missing required AQI columns:

BR001.csv → Missing: PM10 (ug/m3), NH3 (ug/m3)
BR002.csv → Missing: PM10 (ug/m3), NH3 (ug/m3)
BR003.csv → Missing: PM10 (ug/m3), NH3 (ug/m3)
CG001.csv → Missing: Ozone (ug/m3), NH3 (ug/m3)
DL007.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL008.csv → Missing: SO2 (ug/m3)
DL009.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL010.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL011.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL016.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL017.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
DL018.csv → Missing: PM2.5 (ug/m3), PM10 (ug/m3), NH3 (ug/m3)
GJ001.csv → Missing: NH3 (ug/m3)
GJ006.csv → Missing: Ozone (ug/m3), NH3 (ug/m3)
GJ017.csv → Missing: Ozone (ug/m3), NH3 (ug/m3)
HR001.csv → Missing: PM10 (ug/m3), NH3 (ug/m3)
HR004.csv → Missing: NH3 (ug/m3)
HR002.csv → Missing: PM10 (ug/m3), NH3 (ug/m3)
HR003.csv → Missing: NH3 (ug/m3)
HR005.csv → Missing: SO2 (ug/m3), NH3 (ug/m3)
JH002.csv → Missing: NH3 (ug/m3)
KA005.csv → Missing: PM2.5 (ug/m3)

Remove the files from the folder as well as the station record

In [42]:
len([entry for entry in missing_columns_report if 'PM10 (ug/m3)' in entry['missing_columns']])

8

In [35]:
# prompt: check if element exists in an array

print('PM10 (ug/m3)' in [entry for entry in missing_columns_report][0]['missing_columns'])

58